<figure>
  <IMG SRC="https://raw.githubusercontent.com/fmeer/public-files/main/TUlogo.png" WIDTH=200 ALIGN="right">
</figure>

# CIEM5110 Workshop 1: Introduction to pyJive

## Exercise: Bar with elastic support
Here we consider the same elastically supported bar you have seen previously in MUDE:

<center><img src="https://raw.githubusercontent.com/ibcmrocha/public/main/supportedbar.png" alt="beam" width="400"/></center>

but here we ignore body loads and set $EA=10$. We would now like to solve this problem with pyJive.

Following the derivation in the MUDE notebook and referring to the code in `models/barmodel.py`, the original `BarModel` is missing part of its stiffness matrix, related to the support stiffness $k$:

$$ \mathbf{K} = \int \mathbf{B}^TEA\mathbf{B}\,dx + \int \mathbf{N}^Tk\mathbf{N}\,dx $$,

where the first part is already in `BarModel`. Our goal here is to **implement the second part** and perform a **mesh convergence study** by comparing FEM results with the analytical solution above.

### Implement the stiffness matrix
To add functionality to the existing `BarModel`, we define a derived class here. We can make clever use of OOP by first letting the original `BarModel` assemble the first part of the stiffness and using our new class to compute only the second part. 

When inheriting from a base class, all members of the new class that are also in the base class get overwritten. However, we can still call the original methods by using `super()`. In the code below, you can see we call `super().configure(props,globdat)` to call the original `configure` method in `BarModel`. With this, `BarModel` reads the $EA$ parameter for us, and we then only need to read $k$.

Look at the derived class below. Your task is to add the new contribution related to $k$:

$$ \mathbf{K} = \mathbf{K}_\mathrm{super} + \mathbf{K}_k $$

**TIP**: Refer to how $\mathbf{K}_\mathrm{super}$ is assembled by looking at the `_get_matrix()` method of `models/barmodel.py`. Relate this to the story about numerical integration from MUDE.

In [ ]:
import sys
sys.path.append('../../')

from models.barmodel import *

K = 'k'

class SupportedBarModel(BarModel):

    def configure(self, props, globdat):
        super().configure(props, globdat)
        self._k = float(props[K])

    def _get_matrix(self, params, globdat):
        # Add K_super
        super()._get_matrix(params,globdat)
        
        # Add the continuous support contribution
        k = np.array([[self._k]])
        for elem in self._elems:
            inodes = elem.get_nodes()
            idofs = globdat[gn.DOFSPACE].get_dofs(inodes, DOFTYPES[0:self._rank])
            coords = np.stack([globdat[gn.NSET][i].get_coords() for i in inodes], axis=1)[0:self._rank, :]
            sfuncs = self._shape.get_shape_functions()
            weights = self._shape.get_integration_weights(coords)

            elmat = np.zeros((self._dofcount, self._dofcount))
            for ip in range(self._ipcount):
                N = np.zeros((1, self._nodecount))
                N[0, :] = sfuncs[:, ip].transpose()
                
                # elmat += ???

            params[pn.MATRIX0][np.ix_(idofs, idofs)] += elmat

def declare(factory):
    factory.declare_model('SupportedBar', SupportedBarModel)

We now read the input files and change a few things:

In [ ]:
import matplotlib.pyplot as plt

from utils import proputils as pu
import main
from names import GlobNames as gn

props = pu.parse_file('bar.pro')

props['model']['bar']['type'] = 'SupportedBar'

Now look at the figure above and check again what is in `bar.pro` and make all the relevant properties match. You can either do it by modifying `props` in the block below or by directly changing `bar.pro`

### Run the analysis and check the output

Now, you can run the new model. Because we have defined a new model in the notebook, we have to declare this one manually and create the `globdat` object before we call `main.jive`

In [ ]:
import declare

globdat = {}

# default declaration

declare.declare_models(globdat)
declare.declare_modules(globdat)
declare.declare_shapes(globdat)
    
# Additional declaration of model defined in the notebook
    
factory = globdat[gn.MODELFACTORY]
factory.declare_model('SupportedBar',SupportedBarModel)
    
globdat = main.jive(props, globdat)

Because of the distributed supports, it is interesting to plot the displacement field here:

In [ ]:
u = globdat[gn.STATE0]
print(u)

plt.plot(np.linspace(0,3,4),u)

### Perform a mesh refinement study

But how can we know this solution makes any sense? We can try meshes with different numbers of elements and get an idea of how sensitive the solution is to further mesh discretization:

In [ ]:
# Add more meshes here!
number_elements = [3];

def mesher(n):
    dx = 3.0 / n
    with open('newmesh.mesh', 'w') as fmesh:
        fmesh.write('nodes (ID, x, [y], [z])\n')
        for i in range(n + 1):
            fmesh.write('%d %f\n' % (i, i * dx))
        fmesh.write('elements (node#1, node#2, [node#3, ...])\n')
        for i in range(n):
            fmesh.write('%d %d\n' % (i, i + 1))

def run_model(props):            
    globdat = {}
    
    declare.declare_models(globdat)
    declare.declare_modules(globdat)
    declare.declare_shapes(globdat)
    
    factory = globdat[gn.MODELFACTORY]
    factory.declare_model('SupportedBar',SupportedBarModel)

    globdat = main.jive(props, globdat) 
    
    return globdat   

plt.figure()
for ne in number_elements:
    print('\n\nRunning model with',ne,'elements')
    mesher(ne)
    props['init']['mesh']['file'] = 'newmesh.mesh'
    globdat = run_model(props)
    stiff = globdat[gn.MATRIX0]
    solution = globdat[gn.STATE0]
    plt.plot(np.linspace(0,3,ne+1),solution,label=str(ne) + ' elements')

plt.xlabel('Position [m]')
plt.ylabel('Displacement [m]')
plt.legend()
plt.show()

What conclusions can you take? How many elements are enough for this problem?